In [6]:
from inspect import getfullargspec
def fa(**ann):
    """
    Returns a decorator annotating a function with the given annotations.
    This is a trick to support function annotations in Python 2.X.
    """
    def annotate(f):
        fas = getfullargspec(f)
        args = fas.args
        if fas.varargs:
            args.append(fas.varargs)
        if fas.varkw:
            args.append(fas.varkw)
        for argname in ann:
            if argname not in args:
                # raise NameError(
                #    _('Annotating non-existing argument: %s') % argname)
                pass
        f.__annotations__ = ann
        return f
    return annotate

In [7]:
class pairs(object):
    def __init__(self, **kwargs):
        self.vars=kwargs
def def_act(**kwargs):
    def runner():
        print('.................')
        for k,v in kwargs.items():
            print(k,v)
    return runner

In [8]:
@fa(
    patterns_loc=("Path to gazetteer", "positional", None, str),
    text_loc=("Path to Reddit corpus file", "positional", None, str),
    n=("Number of texts to read", "option", "n", int),
    lang=("Language class to initialise", "option", "l", str),
    lang_ex=("Language extension to initialise", "option", def_act(x=1), pairs(value=18)),
)
def fun(patterns_loc, text_loc, n=10000, lang="en", lang_ex="..."):
    print(f"loc: {patterns_loc}, text: {text_loc}, n: {n}, lang: {lang}")

print('.. start')
fun('loc', 'text', 500, 'en')

.. start
loc: loc, text: text, n: 500, lang: en


In [9]:
import inspect
NONE = object()  # sentinel use to signal the absence of a default

def getargspec(callableobj):
    """Given a callable return an object with attributes .args, .varargs,
    .varkw, .defaults. It tries to do the "right thing" with functions,
    methods, classes and generic callables."""
    import inspect
    if inspect.isfunction(callableobj):
        argspec = getfullargspec(callableobj)
    elif inspect.ismethod(callableobj):
        argspec = getfullargspec(callableobj)
        del argspec.args[0]  # remove first argument
    elif inspect.isclass(callableobj):
        if callableobj.__init__ is object.__init__:  # to avoid an error
            argspec = getfullargspec(lambda self: None)
        else:
            argspec = getfullargspec(callableobj.__init__)
        del argspec.args[0]  # remove first argument
    elif hasattr(callableobj, '__call__'):
        argspec = getfullargspec(callableobj.__call__)
        del argspec.args[0]  # remove first argument
    else:
        raise TypeError(_('Could not determine the signature of ') +
                        str(callableobj))
    return argspec

def fcall(obj):
    f = getargspec(obj)
    defaults = f.defaults or ()
    n_args = len(f.args)
    n_defaults = len(defaults)
    alldefaults = (NONE,) * (n_args - n_defaults) + defaults
    func = obj
    for name, default in zip(f.args, alldefaults):
        ann = f.annotations.get(name, ())
        print(ann, default)
        if inspect.isfunction(ann[2]):
            print('\t.. callable')
            ann[2]()
            
    # call function
    extraopts=[]
    if f.varkw:
        kwargs=f.varkw
    else:
        kwargs={}
    print('-'*10)
    obj(*(f.args + extraopts), **kwargs)
    
fcall(fun)

('Path to gazetteer', 'positional', None, <class 'str'>) <object object at 0x10ca16680>
('Path to Reddit corpus file', 'positional', None, <class 'str'>) <object object at 0x10ca16680>
('Number of texts to read', 'option', 'n', <class 'int'>) 10000
('Language class to initialise', 'option', 'l', <class 'str'>) en
('Language extension to initialise', 'option', <function def_act.<locals>.runner at 0x10d5799d8>, <__main__.pairs object at 0x10d57c3c8>) ...
	.. callable
.................
x 1
----------
loc: patterns_loc, text: text_loc, n: n, lang: lang


In [31]:
from inspect import getfullargspec, isfunction
NONE = object()  # sentinel use to signal the absence of a default
def fa_e(**ann):
    def annotate(f):        
        f.__annotations__ = ann
        return f    
    return annotate

class pairs(object):
    def __init__(self, **kwargs):
        self.vars=kwargs
def def_act(**kwargs):
    def runner():
        print('.................')
        for k,v in kwargs.items():
            print(k,v)
    return runner

@fa_e(
    patterns_loc=("Path to gazetteer", "positional", None, str),
    text_loc=("Path to Reddit corpus file", "positional", None, str),
    n=("Number of texts to read", "option", "n", int),
    lang=("Language class to initialise", "option", "l", str),
    lang_ex=("Language extension to initialise", "option", def_act(x=1), pairs(value=18)),
)
def fun_e(patterns_loc, text_loc, n, lang, lang_ex='...'):
    print(f"loc: {patterns_loc}, text: {text_loc}, n: {n}, lang: {lang}, ex: {lang_ex}")
    # print(kwargs)

def fcall_s(obj):
    f = getfullargspec(obj)
    defaults = f.defaults or ()
    n_args = len(f.args)
    n_defaults = len(defaults)
    alldefaults = (NONE,) * (n_args - n_defaults) + defaults
    func = obj
    print('..', obj.__name__)
    for name, default in zip(f.args, alldefaults):
        ann = f.annotations.get(name, ())
        print(ann, default)
        if isfunction(ann[2]):
            print('\t.. callable')
            ann[2]()
            
    # call function
    extraopts=[]
    if f.varkw:
        kwargs=f.varkw
    else:
        kwargs={}
    print('-'*10)
    print('**', f.args, kwargs)
    obj(*(f.args + extraopts), **kwargs)
    
fcall_s(fun_e)
# fcall_s(fun)

.. fun_e
('Path to gazetteer', 'positional', None, <class 'str'>) <object object at 0x1154e3bc0>
('Path to Reddit corpus file', 'positional', None, <class 'str'>) <object object at 0x1154e3bc0>
('Number of texts to read', 'option', 'n', <class 'int'>) <object object at 0x1154e3bc0>
('Language class to initialise', 'option', 'l', <class 'str'>) <object object at 0x1154e3bc0>
('Language extension to initialise', 'option', <function def_act.<locals>.runner at 0x11567f730>, <__main__.pairs object at 0x11548edd8>) ...
	.. callable
.................
x 1
----------
** ['patterns_loc', 'text_loc', 'n', 'lang', 'lang_ex'] {}
loc: patterns_loc, text: text_loc, n: n, lang: lang, ex: lang_ex


In [13]:
import importlib
results = {}
# full_name = package.__name__ + "." + name
full_name='sagas.util.string_util'
results[full_name] = importlib.import_module(full_name)

In [33]:
import pkgutil
import sagas
results = {}
package=sagas.nlu
print(package.__path__)
for loader, name, is_pkg in pkgutil.walk_packages(package.__path__):
    if name.startswith('inspector_'):
        full_name = package.__name__ + "." + name
        results[full_name] = importlib.import_module(full_name)
print(results.keys())

['/Users/xiaofeiwu/jcloud/assets/langs/workspace/rasa/stack/sagas/nlu']
dict_keys(['sagas.nlu.inspector_common', 'sagas.nlu.inspector_fixtures', 'sagas.nlu.inspector_rasa', 'sagas.nlu.inspector_wordnet'])


In [26]:
from sagas.nlu.inspector_common import Inspector
def all_subclasses(cls):
    # type: (Any) -> List[Any]
    """Returns all known (imported) subclasses of a class."""

    return cls.__subclasses__() + [
        g for s in cls.__subclasses__() for g in all_subclasses(s)
    ]

# importlib.import_module('sagas.nlu.inspector_rasa')
all_subclasses(Inspector)

[sagas.nlu.inspectors.DateInspector,
 sagas.nlu.inspectors.NegativeWordInspector,
 sagas.nlu.inspectors.PlainInspector,
 sagas.nlu.inspectors.EntityInspector,
 sagas.nlu.inspector_rasa.RasaInspector,
 sagas.nlu.inspector_wordnet.WordInspector,
 sagas.nlu.inspector_wordnet.PredicateWordInspector,
 sagas.nlu.inspector_wordnet.VerbInspector]

In [35]:
import pkgutil
import sagas.aifuncs
results = {}
package=sagas.aifuncs
print(package.__path__)
for loader, name, is_pkg in pkgutil.walk_packages(package.__path__):
    if name.startswith('rules_'):
        full_name = package.__name__ + "." + name
        results[full_name] = importlib.import_module(full_name)
print(results.keys())

['/Users/xiaofeiwu/jcloud/assets/langs/workspace/rasa/stack/sagas/aifuncs']
dict_keys(['sagas.aifuncs.rules_common'])


In [37]:
from sagas.nlu.ruleset import RuleSet, RuleSets
all_subclasses(RuleSets)

[sagas.nlu.ruleset.BasicRuleSets, sagas.aifuncs.rules_common.CommonRuleSets]